In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    MaxPooling2D,
    Conv2D,
    Dropout,
    BatchNormalization,
)

from src.kerasCNN import *
from src.augmentations.CustomAugmentations import CustomAugmentationsTF

import matplotlib.pyplot as plt
from copy import deepcopy
import pickle

from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm

import numpy as np
import random
import os
os.mkdir("../models")

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
os.environ["PYTHONHASHSEED"] = "42"
keras.utils.set_random_seed(42)

In [ ]:
height, width = 32, 32
batch_size = 64

In [ ]:
# Define the augmentation probabilities
p_dict = {
    # "flip": 0.5,
    # "transpose": 0.5,
    # "gauss_noise": 0.5,
    # "brightness_contrast": 0.5,
    # "hue_saturation_value": 0.5,
}

p_dicts = []
for aug_type in p_dict.keys():
    p_d = {aug_type: 0.5}
    p_dicts.append(p_d)

p_dict = {
    "flip": 0.5,
    # "transpose": 0.5,
    "gauss_noise": 0.5,
    # "brightness_contrast": 0.5,
    # "hue_saturation_value": 0.5,
}
p_dicts.append(p_dict)

p_dicts

In [ ]:
EPOCHS = 50

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for augs in tqdm(p_dicts, desc="Augmentation"):
    aug_str = "_".join([key + str(value) for key, value in augs.items()])
    print(f"Augmentations: {aug_str}")

    for iteration in tqdm(range(5), desc="Iteration"):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/train",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="training",
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/valid",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="validation",
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/test",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=False,
        )

        dnn_model = create_cnn(
            kernel_sizes=[
                3,
                3,
                3,
            ],
            num_filters=[32, 64, 64],
            fc_sizes=[128, 64, 64],
        )

        history = {
            "loss": list(),
            "accuracy": list(),
            "val_loss": list(),
            "val_accuracy": list(),
        }

        print(f"Fitting model {iteration}")
        for epoch in tqdm(range(EPOCHS), desc="Epoch"):
            print(f"Epoch {epoch + 1}/{EPOCHS}")
            custom_aug = CustomAugmentationsTF(augs)
            train_set_aug = train_set.map(lambda x, y: (custom_aug.augment(x), y))

            hist_ = dnn_model.fit(train_set_aug, validation_data=val_set, epochs=1)
            history["loss"].append(hist_.history["loss"])
            history["accuracy"].append(hist_.history["accuracy"])
            history["val_loss"].append(hist_.history["val_loss"])
            history["val_accuracy"].append(hist_.history["val_accuracy"])

        print(f"Predicting model {iteration}")
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f"Accuracy: {accuracy} for model {iteration}")
        class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

        print(f"Saving model {iteration}")
        dnn_model.save(f"../models/BESTClassicCNN_aug_{aug_str}_{iteration}.h5")
        histories.append(deepcopy(history))
        accuracies.append(deepcopy(accuracy))
        class_accuracies.append(deepcopy(class_accuracy))
        conf_mats.append(deepcopy(conf_mat))

    with open(f"results/HISTORY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(histories, f)

    with open(f"results/ACCURACY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(accuracies, f)

    with open(f"results/CLASS_ACCURACY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(class_accuracies, f)

    with open(f"results/CONF_MAT_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(conf_mats, f)